# Regression Models on Ames Real Estate Data

_Author: Alex Chicote (NYC)_

---
<img src="./photo303159.jpeg" style="float: center; margin: 20px; height: 300px">

        Is it possible to predict the price of sale for a real estate unit in Ames IA?

        What are the most important factors to determine it?

## Plan of Action. Feature Engineering

### Steps:



   - [1. Importing all the modules necessary.](#importing)
   - [2. Fixing columns names.](#nulsimes) 
   - [3. Finding and replacing nulls.](#nulsimes)
   - [4. Create dummies for all object columns.](#dummies)
   - [5. Repeat the process for test set.](#second_try)
   - [6. Making sure all features are the same in all sets( except for saleprice and sale_condition.](#killing)
   - [7. Save train and test as a csv.](#vesasaberque)
   

<a id='importing'></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.datasets import load_breast_cancer
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression, LassoCV, RidgeCV, ElasticNetCV

%matplotlib inline

In [6]:
train = pd.read_csv('train.csv')
train['MoSold'].describe()

KeyError: 'MoSold'

<a id='nulsimes'></a>

In [18]:
##loading data(train)
file ='train.csv'
train = pd.read_csv(file)
train.columns = [x.lower().replace(' ', '_') for x in train.columns]
train.shape
numero_train = list(train.select_dtypes(include=['int64', 'float64']).columns)
object_train = list(train.select_dtypes(include=['object']).columns)
##making sure it is correct: must be zero
secure = len(train.columns) - len(numero_train) - len(object_train)
print('You better be zero', secure)
##I do not need to do anything with 
##separating features with nulls by dtypes
numeric_nuls = []
for item in numero_train:
    if train[item].isnull().sum() > 0:
        t = tuple([item, train[item].isnull().sum()])
        numeric_nuls.append(t)
    numeric_nuls.sort(key=lambda x: x[1], reverse=True)
object_nuls = []
for item in object_train:
    if train[item].isnull().sum() > 0:
        t = tuple([item, train[item].isnull().sum()])
        object_nuls.append(t)
    object_nuls.sort(key=lambda x: x[1], reverse=True)
## Feature Engineering and replacing nuls
MEAN = train['lot_frontage'].mean()
train['lot_frontage'] = train['lot_frontage'].fillna(MEAN)
train['garage_yr_blt'] = train['garage_yr_blt'].replace(2207, 2007)
###It is a tuple, IDIOT
for item in object_nuls:
    cosa = item[0]
    if 'garage_' in cosa:
        train[cosa] = train[cosa].fillna(0)
for item in numeric_nuls:
    cosa = item[0]
    if 'garage_' in cosa:
        train[cosa] = train[cosa].fillna(0)
## I will fill with zeros the rest of numerics
for item in numeric_nuls:
    cosa = item[0]
    train[cosa] = train[cosa].fillna(0)
###I fill nuls with zeros for the rest of ojects 
for item in object_nuls:
    melanina = item[0]
    train[melanina] = train[melanina].fillna('0')     
print('You better be zero',(train.isnull().sum() > 0).sum())

You better be zero 0
You better be zero 0


In [19]:
filename ='test.csv'
test = pd.read_csv(filename)
test.columns = [x.lower().replace(' ', '_') for x in test.columns]
test.shape
numero_test = list(test.select_dtypes(include=['int64', 'float64']).columns)

object_test = list(test.select_dtypes(include=['object']).columns)

##making sure it is correct: must be zero
secure = len(test.columns) - len(numero_test) - len(object_test)
print('Has to be zero', secure)

##I do not need to do anything with 
train[numero_test].describe()
numeric_nuls_test = []
for item in numero_test:
    if test[item].isnull().sum() > 0:
        t = tuple([item, test[item].isnull().sum()])
        numeric_nuls_test.append(t)
    numeric_nuls_test.sort(key=lambda x: x[1], reverse=True)

object_nuls_test = []
for item in object_test:
    if test[item].isnull().sum() > 0:
        t = tuple([item, test[item].isnull().sum()])
        object_nuls_test.append(t)
    object_nuls_test.sort(key=lambda x: x[1], reverse=True)

for item in object_nuls_test:
    cosa = item[0]
    if 'garage_' in cosa:
        test[cosa] = test[cosa].fillna(0)
for item in numeric_nuls_test:
    cosa = item[0]
    if 'garage_' in cosa:
        test[cosa] = test[cosa].fillna(0)
MEAN = test['lot_frontage'].mean()
test['lot_frontage'] = test['lot_frontage'].fillna(MEAN)   
###It is a tuple, IDIOT
## I will fill with zeros the rest of numerics and non_numerics
for item in numeric_nuls_test:
    cosa = item[0]
    test[cosa] = test[cosa].fillna(0)
for item in object_nuls_test:
    melanina = item[0]
    test[melanina] = test[melanina].fillna('0')
print('Has to be zero',(test.isnull().sum() > 0).sum())

Has to be zero 0
Has to be zero 0


## Making sure we work with same features in both sets

In [20]:
##Sale_condition feature does not exist in the test dataset so I would eliminate it
TRAIN = set(train.columns)
TEST = set(test.columns)
diff = TRAIN - TEST
print(diff)
DIFF = TEST - TRAIN
print(DIFF)
z = train['sale_condition']
#train.drop('sale_condition', axis=1, inplace = True)

{'saleprice', 'sale_condition'}
set()


<a id='dummies'></a>
## get dummies going

In [22]:
object_train.remove('sale_condition')

In [23]:
test = pd.get_dummies(test, columns=object_test)
train = pd.get_dummies(train, columns=object_train)    
print(train.shape, test.shape)    

(2051, 308) (879, 289)


<a id='killing'></a>
## Finaly, I want eliminate the features that are not in set

In [30]:
##Finaly, I want eliminate the features that are not in set.
TRAIN = set(train.columns)
TEST = set(test.columns)
diff = TRAIN - TEST
diff = diff
print(diff, len(diff))
DIFF = TEST - TRAIN
print(DIFF, len(DIFF))

{'electrical_Mix', 'roof_matl_ClyTile', 'functional_Sal', 'pool_qc_Fa', 'heating_Wall', 'exterior_2nd_Stone', 'roof_matl_Membran', 'misc_feature_TenC', 'misc_feature_Elev', 'functional_Sev', 'neighborhood_GrnHill', 'heating_OthW', 'condition_2_Artery', 'utilities_NoSeWa', 'ms_zoning_A (agr)', 'heating_qc_Po', 'garage_qual_Ex', 'condition_2_RRAn', 'exterior_1st_CBlock', 'bsmt_cond_Ex', 'exterior_1st_Stone', 'condition_2_RRNn', 'saleprice', 'neighborhood_Landmrk', 'exterior_1st_ImStucc', 'condition_2_RRAe', 'pool_qc_Gd', 'bsmt_cond_Po', 'sale_condition'} 29
{'electrical_0', 'kitchen_qual_Po', 'roof_matl_Metal', 'exterior_2nd_Other', 'exterior_1st_PreCast', 'exterior_2nd_PreCast', 'sale_type_VWD', 'roof_matl_Roll', 'mas_vnr_type_CBlock', 'heating_Floor'} 10


In [31]:
diff = list(diff)
print(len(diff))
diff = [c for c in diff if c != 'saleprice']
diff = [c for c in train.columns if c not in diff]
diff.append('sale_condition')
print(len(diff))

29
281


In [33]:
DIFF = list(DIFF)
print(len(DIFF))

DIFF = [c for c in test.columns if c not in DIFF]
print(len(DIFF))


10
279


<a id='vesasaberque'></a>
## Saving final train and final set

In [34]:
##Saving final dataseta
train[diff].to_csv('final_train.csv', index=False, encoding = 'utf-8' )
test[DIFF].to_csv('final_test.csv', index=False, encoding= 'utf-8')